# 1 Quarantine Tables

En este entorno aún **no estamos en producción**, por lo que vamos a trabajar de forma **iterativa y experimental** sobre los datos que han sido enviados a la tabla de **cuarentena**.

El objetivo es **detectar los errores más frecuentes**, **probar correcciones paso a paso** y **evaluar sus efectos** antes de integrarlas definitivamente en el flujo de validación oficial.

A lo largo de este notebook, iremos aplicando transformaciones que conviertan los registros en cuarentena en datos **limpios, consistentes y listos para su reintegración en la capa Silver**.


## 1.1 Dim_Aircraft

Empecemos mostrando los registros que tenemos actualmente en la tabla Dim_Aircraft de Cuarentena

In [0]:
%sql
--drop table if exists sesion1_2.layer30_silver_quarantine.dim_aircraft_quarantine;

In [0]:
%sql
select *
from sesion1_2.layer30_silver_quarantine.dim_aircraft_quarantine

aircraft_id,tail_number,model,manufacturer,engine_model,first_service_dt,status,cycles_total,hours_total,fixed,type_fix,date_fix,fix_version
AC1001,N37173,787-8,Boeing,LEAP-1B,2002-06-24,Retired,3130,-6236.1,null,null,null,null
AC1009,N99732,737-800,Boeing,CFM56-7B,2012-10-04,Active,28990,-55523.5,null,null,null,null
AC1021,N69910,737-800,Boeing,CFM56-7B,2006-09-09,Active,41839,-91357.6,null,null,null,null
AC1061,N93500,737-9,Boeing,GEnx-1B,2021-07-18,Maintenance,24415,-53089.3,null,null,null,null
AC1067,N85417,787-8,Boeing,LEAP-1B,2019-02-11,Active,19008,-41446.5,null,null,null,null
AC1075,N55407,787-8,Boeing,LEAP-1B,2020-02-11,Active,34601,-68569.7,null,null,null,null
AC1000,N11088,Citation CJ3,Cessna,PT6A-114,2007-05-17,Maintenance,27289,59122.7,true,engine_model_from_lookup_by_model,2025-10-09T15:49:07.439Z,1
AC1013,N35551,Citation CJ3,Cessna,PT6A-114,2019-01-31,Maintenance,3413,7318.5,true,engine_model_from_lookup_by_model,2025-10-09T15:49:07.439Z,1
AC1022,N22287,G650,Gulfstream,BR710,2014-04-20,Active,6230,13209.8,true,engine_model_from_lookup_by_model,2025-10-09T15:49:07.439Z,1
AC1026,N71264,737-800,Boeing,CFM56-7B,2022-10-19,Active,34779,75763.0,true,engine_model_from_lookup_by_model,2025-10-09T15:49:07.439Z,1


### 1.1.0 Creamos columnas de arreglos

%md
Para poder **gestionar de forma trazable** los errores y sus correcciones, añadiremos una serie de columnas a la tabla de **cuarentena**.  
Estas columnas nos permitirán registrar tanto el estado de cada registro como la evolución de las correcciones aplicadas a lo largo del tiempo.

A continuación, añadimos las siguientes columnas:

| Columna | Tipo | Descripción |
|----------|------|-------------|
| **fixed** | `BOOLEAN` | Indica si el registro ya ha sido corregido (`TRUE`) o sigue pendiente (`FALSE`). |
| **type_fix** | `STRING` | Describe el tipo de corrección aplicada (por ejemplo: *null_filled*, *format_adjusted*, *id_relinked*, etc.). |
| **date_fix** | `TIMESTAMP` | Fecha y hora en la que se realizó la corrección. Permite hacer seguimiento temporal de los cambios. |
| **fix_version** | `INT` | Número de versión o iteración de la corrección, útil si un mismo registro ha sido corregido más de una vez. |

Con esto, la tabla de cuarentena no solo almacena los registros problemáticos, sino también **su historial de resolución**, lo que facilita el **análisis de calidad de datos** y la **auditoría del proceso de validación**.


``Ejercicio`` Prepara un bloque de codigo añadiendo las columnas de auditoria:

- "fixed" BOOLEAN
- "type_fix" STRING
- "date_fix" TIMESTAMP
- "fix_version" IN

Debe comprobar si estas columnas ya existen en la tabla sesion1_2.layer30_silver_quarantine.dim_aircraft_quarantine.

-> Si existen No hace nada

-> Si no existen crea las columnas

In [0]:
# Prepara el codigo


### 1.1.1 Caso de corrección: incoherencia entre horas y ciclos de vuelo

Los registros correspondientes a aeronaves del fabricante **Boeing** representan vuelos **nacionales**, caracterizados por un ciclo medio de duración comprendido entre **1 hora y 30 minutos y 2 horas y 15 minutos**.

Durante la validación se ha detectado que, en algunos casos, la columna `cycles_total` contiene valores registrados, mientras que la columna `hours_total` aparece vacía.  
Dado que existe un consenso técnico sobre la relación aproximada entre ciclos y horas de vuelo, se acordó **corregir estos registros asignando a `hours_total` el valor `2000 * 2`**.

Esta corrección permite mantener la coherencia interna de los datos y asegurar la consistencia entre ambas métricas operativas.


``Ejercicio: Completa`` los ? o propon otro enfoque

In [0]:
%sql
UPDATE sesion1_2.layer30_silver_quarantine.dim_aircraft_quarantine
SET 
    hours_total = ?? * 2,           -- regla de corrección
    fixed = ??,                             -- marcamos como corregido
    type_fix = 'fix_hours_from_cycles',       -- tipo de arreglo aplicado
    date_fix = ??,                          -- timestamp del arreglo
    fix_version = 1                           -- versión de la regla
WHERE manufacturer = ??
  AND (hours_total ?? )
  AND cycles_total ??;



num_affected_rows
6


In [0]:
%sql
select *
from sesion1_2.layer30_silver_quarantine.dim_aircraft_quarantine


aircraft_id,tail_number,model,manufacturer,engine_model,first_service_dt,status,cycles_total,hours_total,fixed,type_fix,date_fix,fix_version
AC1001,N37173,787-8,Boeing,LEAP-1B,2002-06-24,Retired,3130,-6236.1,null,null,null,null
AC1009,N99732,737-800,Boeing,CFM56-7B,2012-10-04,Active,28990,-55523.5,null,null,null,null
AC1021,N69910,737-800,Boeing,CFM56-7B,2006-09-09,Active,41839,-91357.6,null,null,null,null
AC1061,N93500,737-9,Boeing,GEnx-1B,2021-07-18,Maintenance,24415,-53089.3,null,null,null,null
AC1067,N85417,787-8,Boeing,LEAP-1B,2019-02-11,Active,19008,-41446.5,null,null,null,null
AC1075,N55407,787-8,Boeing,LEAP-1B,2020-02-11,Active,34601,-68569.7,null,null,null,null
AC1000,N11088,Citation CJ3,Cessna,PT6A-114,2007-05-17,Maintenance,27289,59122.7,true,engine_model_from_lookup_by_model,2025-10-09T15:49:07.439Z,1
AC1013,N35551,Citation CJ3,Cessna,PT6A-114,2019-01-31,Maintenance,3413,7318.5,true,engine_model_from_lookup_by_model,2025-10-09T15:49:07.439Z,1
AC1022,N22287,G650,Gulfstream,BR710,2014-04-20,Active,6230,13209.8,true,engine_model_from_lookup_by_model,2025-10-09T15:49:07.439Z,1
AC1026,N71264,737-800,Boeing,CFM56-7B,2022-10-19,Active,34779,75763.0,true,engine_model_from_lookup_by_model,2025-10-09T15:49:07.439Z,1


###  1.1.2 Caso de corrección: valores nulos en *Engine Model*

Se han identificado registros en la tabla de cuarentena con la columna `engine_model` vacía.  
Para estos casos, se deberá **consultar la tabla Silver** correspondiente y **asignar el valor de `engine_model` utilizado para el mismo modelo de aeronave**.


In [0]:
%sql
select *
from sesion1_2.layer30_silver_quarantine.dim_aircraft_quarantine
where engine_model is null
or trim(engine_model) = ''


aircraft_id,tail_number,model,manufacturer,engine_model,first_service_dt,status,cycles_total,hours_total
AC1000,N11088,Citation CJ3,Cessna,null,2007-05-17,Maintenance,27289,59122.7
AC1013,N35551,Citation CJ3,Cessna,null,2019-01-31,Maintenance,3413,7318.5
AC1022,N22287,G650,Gulfstream,null,2014-04-20,Active,6230,13209.8
AC1026,N71264,737-800,Boeing,,2022-10-19,Active,34779,75763.0
AC1029,N12609,E175,Embraer,null,2003-10-14,Active,42335,77812.6
AC1030,N32132,Citation CJ3,Cessna,null,2012-04-05,Active,40234,82337.5
AC1074,N72762,A321-231,Airbus,,2016-07-22,Active,9962,17980.6


- Creamos una vista con las valores unicos de modelo y su engine_model asociado 

``Ejercicio: Completa`` los ? o propon otro enfoque

In [0]:
%sql
-- Ejemplo. Si se prefiere puedes utilizar otro enfoque

CREATE OR REPLACE VIEW sesion1_2.layer30_silver.dim_aircraft_models AS
SELECT 
  ??,
  ?? AS engine_model
FROM sesion1_2.layer30_silver.dim_aircraft
WHERE engine_model ??
  AND trim(engine_model) ??
GROUP BY model;



- Veamos la vista que hemos creado

In [0]:
%sql
SELECT * 
FROM sesion1_2.layer30_silver.dim_aircraft_models
ORDER BY model
LIMIT 50;


model,engine_model
208 Caravan,PT6A-114
737-800,CFM56-7B
737-9,GEnx-1B
787-8,LEAP-1B
787-9,LEAP-1B
A320-214,CFM56-5B
A321-231,Trent-700
A330-200,V2500
A350-900,V2500
Citation CJ3,PT6A-114


- Por ultimo hagamos el merge trayendo a nuestra tabla ``dim_aircraft_quarantine`` los engine_model correspondientes desde la vista creada

In [0]:
%sql
-- Rellenar engine_model en cuarentena a partir de la vista de referencia por model



num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
0,0,0,0


In [0]:
%sql
select *
from sesion1_2.layer30_silver_quarantine.dim_aircraft_quarantine

aircraft_id,tail_number,model,manufacturer,engine_model,first_service_dt,status,cycles_total,hours_total,fixed,type_fix,date_fix,fix_version
AC1001,N37173,787-8,Boeing,LEAP-1B,2002-06-24,Retired,3130,-6236.1,null,null,null,null
AC1009,N99732,737-800,Boeing,CFM56-7B,2012-10-04,Active,28990,-55523.5,null,null,null,null
AC1021,N69910,737-800,Boeing,CFM56-7B,2006-09-09,Active,41839,-91357.6,null,null,null,null
AC1061,N93500,737-9,Boeing,GEnx-1B,2021-07-18,Maintenance,24415,-53089.3,null,null,null,null
AC1067,N85417,787-8,Boeing,LEAP-1B,2019-02-11,Active,19008,-41446.5,null,null,null,null
AC1075,N55407,787-8,Boeing,LEAP-1B,2020-02-11,Active,34601,-68569.7,null,null,null,null
AC1000,N11088,Citation CJ3,Cessna,PT6A-114,2007-05-17,Maintenance,27289,59122.7,true,engine_model_from_lookup_by_model,2025-10-09T15:49:07.439Z,1
AC1013,N35551,Citation CJ3,Cessna,PT6A-114,2019-01-31,Maintenance,3413,7318.5,true,engine_model_from_lookup_by_model,2025-10-09T15:49:07.439Z,1
AC1022,N22287,G650,Gulfstream,BR710,2014-04-20,Active,6230,13209.8,true,engine_model_from_lookup_by_model,2025-10-09T15:49:07.439Z,1
AC1026,N71264,737-800,Boeing,CFM56-7B,2022-10-19,Active,34779,75763.0,true,engine_model_from_lookup_by_model,2025-10-09T15:49:07.439Z,1


### 1.1.3 Join a Tabla Principal

%md
### 🔄 Reintegración de registros corregidos en la capa Silver

Una vez completadas las correcciones en la tabla de **cuarentena**, los registros marcados como `fixed = TRUE` pueden ser **reincorporados** a la tabla principal de la capa **Silver**.  

Para ello, se realiza una operación de tipo **MERGE**, que compara ambas tablas (`Silver` y `Quarantine`) en base a la clave primaria `aircraft_id`.  
Este proceso actualiza los registros ya existentes y añade aquellos que aún no estaban presentes en la tabla destino.

El flujo resultante es:

1. **Seleccionar** los registros corregidos en cuarentena (`fixed = TRUE`).  
2. **Actualizar** en la tabla Silver los campos modificados.  
3. **Insertar** los nuevos registros que aún no existían.  

De esta forma, los datos recuperados pasan a formar parte de la tabla **`dim_aircraft`** validada, garantizando trazabilidad, control de versiones y consistencia con el proceso de validación general.


In [0]:
%sql
MERGE INTO sesion1_2.layer30_silver.dim_aircraft AS tgt
USING (
  SELECT
    aircraft_id, tail_number, model, manufacturer,
    engine_model, first_service_dt, status,
    cycles_total, hours_total
  FROM sesion1_2.layer30_silver_quarantine.dim_aircraft_quarantine
  WHERE fixed = TRUE
) AS src
ON tgt.aircraft_id = src.aircraft_id

WHEN MATCHED THEN UPDATE SET
  tgt.tail_number      = src.tail_number,
  tgt.model            = src.model,
  tgt.manufacturer     = src.manufacturer,
  tgt.engine_model     = src.engine_model,
  tgt.first_service_dt = src.first_service_dt,
  tgt.status           = src.status,
  tgt.cycles_total     = src.cycles_total,
  tgt.hours_total      = src.hours_total

WHEN NOT MATCHED THEN INSERT (
  aircraft_id, tail_number, model, manufacturer,
  engine_model, first_service_dt, status,
  cycles_total, hours_total
) VALUES (
  src.aircraft_id, src.tail_number, src.model, src.manufacturer,
  src.engine_model, src.first_service_dt, src.status,
  src.cycles_total, src.hours_total
);


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
7,0,0,7


In [0]:
%sql
DELETE FROM sesion1_2.layer30_silver_quarantine.dim_aircraft_quarantine
WHERE fixed = TRUE;

num_affected_rows
7


## 1.2 facts_sensors


Tras haber completado la recuperación y actualización de los registros en **`dim_aircraft`**, continuaremos con la revisión de la siguiente tabla del flujo.  
El objetivo es **aplicar la misma lógica de análisis, corrección y reintegración**, adaptándola a las particularidades de cada dataset.

En esta nueva sección, identificaremos los principales errores detectados, propondremos las reglas de corrección y prepararemos los datos para su posterior promoción a la capa **Silver**.


In [0]:
%sql
select *
from sesion1_2.layer30_silver_quarantine.fact_engine_sensor_quarantine

reading_id,aircraft_id,event_ts,event_date,engine_pos,phase_of_flight,altitude_ft,ias_kts,egt_c,n1_pct,fuel_flow_kg_h,vib_ips,fixed,type_fix,date_fix,fix_version
R100038,AC1065,2023-03-22T22:24:00.000Z,2023-02-05T13:55:00.000Z,1,Takeoff,54639,430,834.0,82.6,2457.1,1.39,null,null,null,null
R100086,AC1021,2023-04-30T07:46:00.000Z,2023-02-01T17:51:00.000Z,2,Climb,18342,688,701.1,54.1,1480.0,0.41,null,null,null,null
R100086,AC1011,2023-01-31T23:45:00.000Z,2023-01-03T14:21:00.000Z,1,Climb,20002,694,663.8,21.9,4677.1,0.86,null,null,null,null
R100277,AC1000,2023-03-08T08:56:00.000Z,2023-06-28T17:44:00.000Z,2,Climb,56291,331,487.9,34.6,4174.6,0.92,null,null,null,null
R100400,AC1022,2023-04-12T22:56:00.000Z,2023-06-15T05:09:00.000Z,1,Descent,58560,475,785.3,36.5,1300.7,0.38,null,null,null,null
R100554,AC1026,2023-01-19T14:11:00.000Z,2023-03-22T12:44:00.000Z,1,Climb,21867,631,785.0,92.1,2219.2,1.38,null,null,null,null
R100961,AC1071,2023-06-08T01:35:00.000Z,2023-05-19T11:07:00.000Z,2,Climb,11985,646,386.1,67.4,4926.0,1.45,null,null,null,null
R100712,AC1058,2023-04-04T16:22:00.000Z,2023-02-10T07:26:00.000Z,1,Cruise,53791,445,885.8,76.3,3540.8,0.35,null,null,null,null
R100717,AC1016,2023-04-15T08:30:00.000Z,2023-05-03T19:11:00.000Z,1,Climb,7143,638,629.3,28.1,3513.6,1.15,null,null,null,null
R100895,AC1069,2023-04-29T19:35:00.000Z,2023-04-06T20:48:00.000Z,2,Taxi,58338,469,443.6,29.6,5109.3,0.88,null,null,null,null


### 1.2.0 Creamos columnas de trazabilidad de fallos

de la misma manera creamos las mismas columnas que nos ayuden con la trazabilidad de errores

In [0]:
table = "sesion1_2.layer30_silver_quarantine.fact_engine_sensor_quarantine"
existing = {f.name.lower() for f in spark.table(table).schema}

to_add = []
if "fixed" not in existing:        to_add.append("fixed BOOLEAN")
if "type_fix" not in existing:     to_add.append("type_fix STRING")
if "date_fix" not in existing:     to_add.append("date_fix TIMESTAMP")
if "fix_version" not in existing:  to_add.append("fix_version INT")

if to_add:
    spark.sql(f"ALTER TABLE {table} ADD COLUMNS ({', '.join(to_add)})")

### 1.2.1 Caso de corrección: altitudes anómalas

Se han identificado registros con valores de **altitud superiores a 50.000 pies**, considerados atípicos para operaciones estándar.  
Sin embargo, ciertos modelos específicos de aeronaves sí pueden alcanzar dichas altitudes, por lo que **estos casos se consideran válidos**.

Por tanto, la regla de corrección consiste en:
- **Marcar como erróneos** los registros con `altitude > 50000`,  
- **Exceptuando** aquellos cuyo `model` pertenezca al grupo de aeronaves `737-800`.

Esta validación permite mantener la integridad de los datos sin penalizar los comportamientos esperables en modelos concretos.


In [0]:
%sql
-- Propon un enfoque para resolver el 1.2.1


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
6,6,0,0


### 1.2.2 Caso de corrección: validación de velocidad indicada (*IAS_KTS*)

La columna `ias_kts` representa la **velocidad indicada en nudos** (*Indicated Air Speed*).  
En nuestro pipeline, el rango operativo definido se encuentra entre **0 y 600 nudos**, considerándose cualquier valor fuera de ese intervalo como anómalo.

No obstante, se han identificado **excepciones operativas** en las que ciertas combinaciones de *fase de vuelo* y *modelo de aeronave* pueden justificar valores fuera de rango.  

Por tanto, la regla de corrección es la siguiente:
- **Exceptuar** aquellos casos en los que la columna `flight_phase` sea `"CLIMB"` o `"DESCENT"`

De esta manera, se preserva la coherencia del dataset sin penalizar comportamientos técnicamente válidos bajo condiciones específicas.


In [0]:
%sql
-- Propon un enfoque para resolver el 1.2.2



num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
0,0,0,0


### 1.2.3 Join a Tabla Principal

In [0]:
%sql
-- 1) Upsert de filas corregidas (fixed=TRUE) desde la cuarentena a silver
MERGE INTO sesion1_2.layer30_silver.fact_engine_sensor AS tgt
USING (
  SELECT
    reading_id,
    aircraft_id,
    event_ts,
    event_date,
    engine_pos,
    phase_of_flight,
    altitude_ft,
    ias_kts,
    egt_c,
    n1_pct,
    fuel_flow_kg_h,
    vib_ips
  FROM sesion1_2.layer30_silver_quarantine.fact_engine_sensor_quarantine
  WHERE fixed = TRUE
) AS src
ON tgt.reading_id = src.reading_id

WHEN MATCHED THEN UPDATE SET
  tgt.aircraft_id     = src.aircraft_id,
  tgt.event_ts        = src.event_ts,
  tgt.event_date      = src.event_date,
  tgt.engine_pos      = src.engine_pos,
  tgt.phase_of_flight = src.phase_of_flight,
  tgt.altitude_ft     = src.altitude_ft,
  tgt.ias_kts         = src.ias_kts,
  tgt.egt_c           = src.egt_c,
  tgt.n1_pct          = src.n1_pct,
  tgt.fuel_flow_kg_h  = src.fuel_flow_kg_h,
  tgt.vib_ips         = src.vib_ips

WHEN NOT MATCHED THEN INSERT (
  reading_id, aircraft_id, event_ts, event_date, engine_pos, phase_of_flight,
  altitude_ft, ias_kts, egt_c, n1_pct, fuel_flow_kg_h, vib_ips
) VALUES (
  src.reading_id, src.aircraft_id, src.event_ts, src.event_date, src.engine_pos, src.phase_of_flight,
  src.altitude_ft, src.ias_kts, src.egt_c, src.n1_pct, src.fuel_flow_kg_h, src.vib_ips
);

-- 2) (Opcional) limpiar cuarentena de lo ya aplicado
DELETE FROM sesion1_2.layer30_silver_quarantine.fact_engine_sensor_quarantine
WHERE fixed = TRUE;


num_affected_rows
6
